In [1]:
#--------- QLAP ---------#
import pandas as pd
import os
import snowflake.sqlalchemy as snowsa
import sqlalchemy as sa
from pathlib import Path
import csv
import sqlparse
import ast

In [3]:
engine = sa.create_engine(
    snowsa.URL(
        account="vestiairecollective.eu-west-1",
        user=os.environ['DBT_USER'],
        password=os.environ['DBT_PASSWORD'],
#         warehouse=os.environ['WAREHOUSE'],
        warehouse='SERVICE_WH_DEFAULT',
        database=os.environ['DATABASE']
    )
)

def get_data_from_snow(query):
    df = pd.read_sql(query, con=engine)
    return df

In [5]:
fp = "/Users/phillip.beattie/Gitlab/lots/qlap"
sql_file = os.path.join(fp, "fct_query_history_lin_reg.sql")
sql = Path(sql_file).read_text()
csv_file = os.path.join(fp, "fct_query_history_lin_reg.csv")

In [61]:
print(csv_file)

/Users/phillip.beattie/Gitlab/lots/qlap/fct_query_history_lin_reg.csv


In [15]:
if os.path.isfile(csv_file):
    df = pd.read_csv(csv_file)
else:
    df = get_data_from_snow(sql)
#     df = df.replace(r'\n',' ', regex=True)
    df.query_text = df.query_text.replace(r'\n|"',' ', regex=True)
    df.tables_scan_list = df.tables_scan_list.replace(r'\n',' ', regex=True)
#     to_replace=r'^ba.$'
    df.to_csv(fp + '/fct_query_history_lin_reg.csv', quoting=csv.QUOTE_MINIMAL)

df.head()

,query_id,query_text,join_list,tables_scan_list,aggregate_list,query_type,query_load_percent,user_name,warehouse_size,total_elapsed_time,...,gb_spilled_to_local_storage,gb_spilled_to_remote_storage,gb_written,gb_written_to_result,rows_produced,rows_inserted,rows_deleted,rows_updated,rows_unloaded,execution_status
0,01a815ab-3201-fbbf-0000-27695f99ba56,INSERT INTO DWH_OPS.#Tableau_101782_1_Connect_...,[],"[\n ""DWH_OPS.#Tableau_101782_1_Connect_CheckC...",[],INSERT,25.0,PAUL.PROVOST@VESTIAIRECOLLECTIVE.COM,Medium,0.280,...,0.0,0.0,9.536740e-07,0.0,1.0,1,0,0,0,SUCCESS
1,01a8165b-3201-fbc5-0000-27695f9ecb9a,INSERT INTO DWH_PRD.#Tableau_3_1_Connect_Check...,[],"[\n ""DWH_PRD.#Tableau_3_1_Connect_CheckCreate...",[],INSERT,50.0,PAUL.PROVOST@VESTIAIRECOLLECTIVE.COM,Small,0.391,...,0.0,0.0,9.536740e-07,0.0,1.0,1,0,0,0,SUCCESS
2,01a815b0-3201-fbc4-0000-27695f9a0042,SELECT * FROM DWH_INT.FCT_VC_INT_BS_MESSAGE_NE...,[],"[\n ""DWH_INT.FCT_VC_INT_BS_MESSAGE_NEW""\n]",[],SELECT,100.0,CLARA.MICHEL@VESTIAIRECOLLECTIVE.COM,Small,0.745,...,0.0,0.0,0.000000e+00,0.0,300.0,0,0,0,0,SUCCESS
3,01a81605-3201-fbc5-0000-27695f9c632e,SELECT TO_DATE(DATE_PUSH_SENT) AS DATE...,"[\n ""JOIN""\n]","[\n ""DWH_CRM.SUM_VC_CRM_BRAZE_PUSH_NOTIFICATI...","[\n ""GROUP BY""\n]",SELECT,NaN,ZHIXUAN.FANG@VESTIAIRECOLLECTIVE.COM,None,0.134,...,0.0,0.0,0.000000e+00,0.0,NaN,0,0,0,0,SUCCESS
4,01a815ec-3201-fbf1-0000-27695f9b7f06,insert into PRODUCTION.DWH_CRM.FCT_VC_CRM_BRAZ...,[],"[\n ""DWH_CRM.FCT_VC_CRM_BRAZE_API_UPDATE_CUST...",[],INSERT,100.0,CLARA.MICHEL@VESTIAIRECOLLECTIVE.COM,Medium,81.667,...,0.0,0.0,1.293025e-01,0.0,3045233.0,3037692,0,0,0,SUCCESS


In [18]:
df_tables = df[['query_id', 'tables_scan_list']].copy()
df_tables.dtypes
df_tables['tables_scan_list'] = df_tables['tables_scan_list'].apply(lambda x: eval(x.strip()))
df_tables = df_tables.rename(columns={'tables_scan_list': 'table_name'})
df_tables = df_tables.explode('table_name')
df_tables.head()

,query_id,table_name
0,01a815ab-3201-fbbf-0000-27695f99ba56,DWH_OPS.#Tableau_101782_1_Connect_CheckCreateT...
1,01a8165b-3201-fbc5-0000-27695f9ecb9a,DWH_PRD.#Tableau_3_1_Connect_CheckCreateTempTa...
2,01a815b0-3201-fbc4-0000-27695f9a0042,DWH_INT.FCT_VC_INT_BS_MESSAGE_NEW
3,01a81605-3201-fbc5-0000-27695f9c632e,DWH_CRM.SUM_VC_CRM_BRAZE_PUSH_NOTIFICATION
3,01a81605-3201-fbc5-0000-27695f9c632e,DWH_MKT.DIM_VC_MKT_CRM_CAMPAIGNS_CATEGORY


In [19]:
df_storage = get_data_from_snow(
"""
select SCHEMA_NAME || '.' || TABLE_NAME as table_name
     , TB_ACTIVE * 1000                 as storage_gb
from meta.OPTIMIZATION.TABLE_STORAGE
where DATABASE_NAME = 'PRODUCTION'
"""
)
df_storage.head()

,table_name,storage_gb
0,DWH_MKT.FCT_VC_MKT_ATTRIBUTION_PRE_RESULTS,0.337
1,DWH_INT.FCT_VC_INT_MMAO_RLT,1.593
2,STAGING.FCT_VC_OPS_STAGING_CURATION_EVENTS_RLT,55.345
3,STAGING.DIM_VC_STAGING_PRODUCT_RANK,1.358
4,STAGING.DIM_VC_PRD_PRODUCT_CURATION_DATETIMES,2.959


In [119]:
df_merge

,query_id,table_name,storage_gb
0,01a80fe6-3201-fb3e-0000-27695f6db0fe,DWH_OPS.FCT_VC_OPS_CURATION_EVENTS_RLT,64.798
1,01a81267-3201-fbbf-0000-27695f828ab2,DWH_OPS.FCT_VC_OPS_CURATION_EVENTS_RLT,64.798
2,01a81267-3201-fbbf-0000-27695f828ab2,DWH_OPS.FCT_VC_OPS_CURATION_EVENTS_RLT,64.798
3,01a81267-3201-fbbf-0000-27695f828ab2,DWH_OPS.FCT_VC_OPS_CURATION_EVENTS_RLT,64.798
4,01a810bb-3201-fb63-0000-27695f765a42,DWH_OPS.FCT_VC_OPS_CURATION_EVENTS_RLT,64.798
...,...,...,...
24475,01a805f7-3201-f840-0000-27695f28491a,DWH_FIN.DIM_VC_FIN_PC2_CURATION_COST,1.313
24476,01a814fa-3201-fbc4-0000-27695f9326ce,DWH_USR.DIM_VC_USR_CRM_NL_SEGMENTATION,0.607
24477,01a80354-3201-f93e-0000-27695f19cb02,DWH_OPS.FCT_VC_OPS_RETURNS,0.286
24478,01a80af2-3201-faad-0000-27695f4b9166,DWH_OPS.FCT_VC_OPS_CHAT_BOT,1.660


In [21]:
df_merge = df_tables.merge(df_storage)
df_agg = df_merge.groupby(by=['query_id'], as_index=False).sum()
df_agg.rename(columns={'storage_gb': 'total_scan_potential_gb'}, inplace=True)
df_deduped = df_merge.drop_duplicates().groupby(by=['query_id'], as_index=False).sum()
df_deduped.rename(columns={'storage_gb': 'distinct_scan_gb'}, inplace=True)
df_scan_info = df_deduped.merge(df_agg)
# print(df_deduped.head())
df_scan_info.head()

,query_id,distinct_scan_gb,total_scan_potential_gb
0,01a801a4-3201-f840-0000-27695f11fb02,64.267,64.267
1,01a801a4-3201-f840-0000-27695f11fc5a,64.331,64.331
2,01a801a4-3201-f883-0000-27695f12049a,64.331,64.331
3,01a801a4-3201-f93d-0000-27695f121226,28.947,28.947
4,01a801a4-3201-f93d-0000-27695f12133a,64.331,64.331


In [22]:
df = df.merge(df_scan_info)
df.head()

,query_id,query_text,join_list,tables_scan_list,aggregate_list,query_type,query_load_percent,user_name,warehouse_size,total_elapsed_time,...,gb_written,gb_written_to_result,rows_produced,rows_inserted,rows_deleted,rows_updated,rows_unloaded,execution_status,distinct_scan_gb,total_scan_potential_gb
0,01a815b0-3201-fbc4-0000-27695f9a0042,SELECT * FROM DWH_INT.FCT_VC_INT_BS_MESSAGE_NE...,[],"[ ""DWH_INT.FCT_VC_INT_BS_MESSAGE_NEW"" ]",[],SELECT,100.0,CLARA.MICHEL@VESTIAIRECOLLECTIVE.COM,Small,0.745,...,0.000000,0.0,300.0,0,0,0,0,SUCCESS,28.625,28.625
1,01a81605-3201-fbc5-0000-27695f9c632e,SELECT TO_DATE(DATE_PUSH_SENT) AS DATE...,"[\n ""JOIN""\n]","[ ""DWH_CRM.SUM_VC_CRM_BRAZE_PUSH_NOTIFICATIO...","[\n ""GROUP BY""\n]",SELECT,NaN,ZHIXUAN.FANG@VESTIAIRECOLLECTIVE.COM,None,0.134,...,0.000000,0.0,NaN,0,0,0,0,SUCCESS,1.801,1.801
2,01a815ec-3201-fbf1-0000-27695f9b7f06,insert into PRODUCTION.DWH_CRM.FCT_VC_CRM_BRAZ...,[],"[ ""DWH_CRM.FCT_VC_CRM_BRAZE_API_UPDATE_CUSTO...",[],INSERT,100.0,CLARA.MICHEL@VESTIAIRECOLLECTIVE.COM,Medium,81.667,...,0.129303,0.0,3045233.0,3037692,0,0,0,SUCCESS,98.988,98.988
3,01a815bc-3201-fbc4-0000-27695f9a0d8e,SELECT DATE_SESSION date --- ALREADY D...,"[\n ""JOIN""\n]","[ ""DWH_TRK.FCT_VC_TRK_DAU_TRAFFICSOURCE"", ...","[\n ""DISTINCT"",\n ""GROUP BY""\n]",SELECT,NaN,ZHIXUAN.FANG@VESTIAIRECOLLECTIVE.COM,None,0.131,...,0.000000,0.0,NaN,0,0,0,0,SUCCESS,59.069,59.069
4,01a81583-3201-fbc4-0000-27695f981b6a,"SELECT DATE_AGGREGATION AS DATE_EVENT, ...","[\n ""JOIN"",\n ""JOIN""\n]","[ ""DWH_MKT.SUM_VC_MKT_ATTRIBUTION_PERFORMANC...","[\n ""group by""\n]",SELECT,NaN,NITHESH.RAMANNA@VESTIAIRECOLLECTIVE.COM,None,0.114,...,0.000000,0.0,NaN,0,0,0,0,SUCCESS,2.318,2.318


In [35]:
from ast import literal_eval

In [37]:
df.join_list.apply(lambda x: len(literal_eval(x)))

0        0
1        1
2        0
3        1
4        2
        ..
10990    1
10991    0
10992    3
10993    0
10994    0
Name: join_list, Length: 10995, dtype: int64

In [38]:
df['nb_joins'] = df.join_list.apply(lambda x: len(literal_eval(x)))
df['nb_aggregations'] = df.aggregate_list.apply(lambda x: len(literal_eval(x)))

In [39]:
df.head()

,query_id,query_text,join_list,tables_scan_list,aggregate_list,query_type,query_load_percent,user_name,warehouse_size,total_elapsed_time,...,rows_produced,rows_inserted,rows_deleted,rows_updated,rows_unloaded,execution_status,distinct_scan_gb,total_scan_potential_gb,nb_joins,nb_aggregations
0,01a815b0-3201-fbc4-0000-27695f9a0042,SELECT * FROM DWH_INT.FCT_VC_INT_BS_MESSAGE_NE...,[],"[ ""DWH_INT.FCT_VC_INT_BS_MESSAGE_NEW"" ]",[],SELECT,100.0,CLARA.MICHEL@VESTIAIRECOLLECTIVE.COM,Small,0.745,...,300.0,0,0,0,0,SUCCESS,28.625,28.625,0,0
1,01a81605-3201-fbc5-0000-27695f9c632e,SELECT TO_DATE(DATE_PUSH_SENT) AS DATE...,"[\n ""JOIN""\n]","[ ""DWH_CRM.SUM_VC_CRM_BRAZE_PUSH_NOTIFICATIO...","[\n ""GROUP BY""\n]",SELECT,NaN,ZHIXUAN.FANG@VESTIAIRECOLLECTIVE.COM,None,0.134,...,NaN,0,0,0,0,SUCCESS,1.801,1.801,1,1
2,01a815ec-3201-fbf1-0000-27695f9b7f06,insert into PRODUCTION.DWH_CRM.FCT_VC_CRM_BRAZ...,[],"[ ""DWH_CRM.FCT_VC_CRM_BRAZE_API_UPDATE_CUSTO...",[],INSERT,100.0,CLARA.MICHEL@VESTIAIRECOLLECTIVE.COM,Medium,81.667,...,3045233.0,3037692,0,0,0,SUCCESS,98.988,98.988,0,0
3,01a815bc-3201-fbc4-0000-27695f9a0d8e,SELECT DATE_SESSION date --- ALREADY D...,"[\n ""JOIN""\n]","[ ""DWH_TRK.FCT_VC_TRK_DAU_TRAFFICSOURCE"", ...","[\n ""DISTINCT"",\n ""GROUP BY""\n]",SELECT,NaN,ZHIXUAN.FANG@VESTIAIRECOLLECTIVE.COM,None,0.131,...,NaN,0,0,0,0,SUCCESS,59.069,59.069,1,2
4,01a81583-3201-fbc4-0000-27695f981b6a,"SELECT DATE_AGGREGATION AS DATE_EVENT, ...","[\n ""JOIN"",\n ""JOIN""\n]","[ ""DWH_MKT.SUM_VC_MKT_ATTRIBUTION_PERFORMANC...","[\n ""group by""\n]",SELECT,NaN,NITHESH.RAMANNA@VESTIAIRECOLLECTIVE.COM,None,0.114,...,NaN,0,0,0,0,SUCCESS,2.318,2.318,2,1


In [52]:
print(df_sql.iloc[0])

query_text    SELECT CAST(FCT_VC_SLS_ORDER_PRODUCT.DATE_SOLD...
Name: 0, dtype: object
